In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [7]:
df=pd.read_csv('24-hours testset.csv')

In [8]:
df=pd.DataFrame(df[['timestamp','day_of_week','hour_of_day','is_weekend','temperature','is_holiday','solar_generation']])

In [9]:
df['timestamp']=pd.to_datetime(df['timestamp'])

In [12]:
def assign_compensation_method(row):
    if row['is_holiday'] == 1:
        return 'Lower Power Supply'
    elif row['solar_generation'] > 200:
        return 'Increased Renewable Energy Integration'
    elif row['hour_of_day'] >= 18 and row['hour_of_day'] <= 22:
        return 'Peak Load Shifting'
    elif row['temperature'] > 35:
        return 'Increased Power Supply'
    else:
        return 'Normal Operation'

df['compensation_method'] = df.apply(assign_compensation_method, axis=1)

print(df.head(10))


            timestamp  day_of_week  hour_of_day  is_weekend  temperature  \
0 2024-01-01 00:00:00            0            0           0    28.534249   
1 2024-01-01 01:00:00            0            1           0    18.140664   
2 2024-01-01 02:00:00            0            2           0    19.539847   
3 2024-01-01 03:00:00            0            3           0    17.519129   
4 2024-01-01 04:00:00            0            4           0    18.255590   
5 2024-01-01 05:00:00            0            5           0    13.403410   
6 2024-01-01 06:00:00            0            6           0    10.759615   
7 2024-01-01 07:00:00            0            7           0    18.029827   
8 2024-01-01 08:00:00            0            8           0     4.909159   
9 2024-01-01 09:00:00            0            9           0    18.700610   

   is_holiday  solar_generation                     compensation_method  
0           0          0.786249                        Normal Operation  
1           0  

In [13]:
df.head()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation,compensation_method
0,2024-01-01 00:00:00,0,0,0,28.534249,0,0.786249,Normal Operation
1,2024-01-01 01:00:00,0,1,0,18.140664,0,7.331477,Normal Operation
2,2024-01-01 02:00:00,0,2,0,19.539847,0,9.877753,Normal Operation
3,2024-01-01 03:00:00,0,3,0,17.519129,0,1.215103,Normal Operation
4,2024-01-01 04:00:00,0,4,0,18.255590,0,2.476539,Normal Operation


In [14]:
df.to_csv('24-hours dataset(old).csv', index=False)

from IPython.display import FileLink

FileLink('24-hours dataset(old).csv')


C:\Users\Rudra\JUPYTER_ML\24-hours dataset(old).csv

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
!pip uninstall numpy
!pip list


In [ ]:
X = df[['temperature', 'solar_generation', 'hour_of_day', 'load', 'is_holiday']]
y=df['compensation_method']
X_reg = df[['temperature', 'solar_generation', 'hour_of_day', 'is_holiday']]
y_reg = df['load']
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
rf_reg.fit(X_reg_train, y_reg_train)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
y_pred = rf_clf.predict(X_test)
class_accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Model Accuracy: {class_accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Make predictions on the test set for regression (peak supply)
y_reg_pred = rf_reg.predict(X_reg_test)
reg_mse = mean_squared_error(y_reg_test, y_reg_pred)
print(f"Regression Model Mean Squared Error: {reg_mse:.2f}")
print(confusion_matrix(y_test, y_pred))

In [ ]:
def predict_for_date(date, temperature, solar_generation, hour_of_day, is_holiday, rf_clf, rf_reg, le):
    new_data_class = pd.DataFrame({
        'temperature': [temperature],
        'solar_generation': [solar_generation],
        'hour_of_day': [hour_of_day],
        'load': [0],
        'is_holiday': [is_holiday]
    })

    new_data_reg = pd.DataFrame({
        'temperature': [temperature],
        'solar_generation': [solar_generation],
        'hour_of_day': [hour_of_day],
        'is_holiday': [is_holiday]
    })
    
    predicted_class_label = rf_clf.predict(new_data_class)
    predicted_compensation_method = le.inverse_transform(predicted_class_label)
    predicted_peak_supply = rf_reg.predict(new_data_reg)

    return {
        'date': date,
        'hour_of_day': hour_of_day,
        'compensation_method': predicted_compensation_method[0],
        'peak_supply_needed': predicted_peak_supply[0]
    }

def generate_24_hour_predictions(date, temperature, solar_generation, is_holiday, rf_clf, rf_reg, le):

    results = []

    for hour in range(24):
        prediction = predict_for_date(date, temperature, solar_generation, hour, is_holiday, rf_clf, rf_reg, le)
        results.append(prediction)
    
    df_predictions = pd.DataFrame(results)
    
    return df_predictions

date = '2024-05-17'
temperature = 27
solar_generation = 50
is_holiday = 1


df_24_hour_predictions = generate_24_hour_predictions(date, temperature, solar_generation, is_holiday, rf_clf, rf_reg, le)

print(df_24_hour_predictions)


In [ ]:
def plot_predictions(df_predictions):
    plt.figure(figsize=(14, 7))

    sns.lineplot(data=df_predictions, x='hour_of_day', y='peak_supply_needed', marker='o', color='blue', label='Peak Supply Needed (MW)')
    
    sns.lineplot(data=df, x='hour_of_day', y='load', palette='Set1')

    plt.title('Hourly Predictions for {}'.format(df_predictions['date'][0]))
    plt.xlabel('Hour of the Day')
    plt.ylabel('Peak Supply Needed (MW)')
    plt.legend(title='Compensation Method')
    plt.grid(True)
    plt.show()

plot_predictions(df_24_hour_predictions)

In [ ]:
def plot_predictions(df_predictions):
    plt.figure(figsize=(14, 7))

    sns.lineplot(data=df_predictions, x='hour_of_day', y='peak_supply_needed', marker='o', color='blue', label='Peak Supply Needed (MW)')
    
    sns.scatterplot(data=df_predictions, x='hour_of_day', y='peak_supply_needed', hue='compensation_method', palette='Set2', s=100, legend='full', marker='o')

    plt.title('Hourly Predictions for {}'.format(df_predictions['date'][0]))
    plt.xlabel('Hour of the Day')
    plt.ylabel('Peak Supply Needed (MW)')
    plt.legend(title='Compensation Method')
    plt.grid(True)
    plt.show()

plot_predictions(df_24_hour_predictions)